In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import math
%matplotlib inline

import sys
sys.path.insert(0, '../../scripts')

import stft_zoom, display, detect_musical_regions
from util import *
import mappings
import pickle
import PIL
import IPython.display
from classes import SingleResSpectrogram, MultiResSpectrogram

/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
sys.path.insert(0, '../../scripts/mauricio_solutions/')
import lukin_todd, swgm, local_sparsity, util_m

In [3]:
%load_ext line_profiler

In [4]:
def calc_kernel_size(window_lengths, energy=False):
    if energy:
        P = 2
    else:
        P = 5
    f_size = P * window_lengths[-1]/window_lengths[0]
    if not f_size % 2 > 0:
        f_size += 1
    
    return [int(f_size), 5]

def LT(y, window_lengths, kernel):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return lukin_todd.lukin_todd(specs, kernel)

def swgm_time(y, window_lengths):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return swgm.SWGM(specs)

def SLS_time(y, window_lengths, kernel_anal, kernel_energy):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return local_sparsity.smoothed_local_sparsity(specs, kernel_anal, kernel_energy)

def our_solution(y, res, kernel, model, sr=44100):
    n_fft = 2048
    spec = np.abs(librosa.stft(y, n_fft=n_fft))
    time_span = [0,len(y)/sr]
    x_axis, y_axis = stft_zoom.get_axes_values(sr, 0, time_span, spec.shape) 
    base_spec = SingleResSpectrogram(spec, x_axis, y_axis)
    multires_spec = MultiResSpectrogram(base_spec)

    indices, original_shape = detect_musical_regions.detect_musical_regions(model, spec, mode='threshold', pct_or_threshold=0.8)
    to_be_refined = detect_musical_regions.musical_regions_to_ranges(indices, original_shape, x_axis, y_axis, kernel)

    stft_zoom.set_signal_bank(y,kernel)

    for subregion in to_be_refined:
        freq_range = subregion[0]
        time_range = subregion[1]
        spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=res)
        refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis)
        multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=1)
        
    return multires_spec

In [5]:
file_name = '../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.wav'
y, sr = librosa.load(file_name, sr=44100)
y = y[:44100*30]

In [6]:
model_200 = pickle.load(open('../renyi_shannon_prollharm_200.sav', 'rb'))
model_500 = pickle.load(open('../renyi_shannon_prollharm_500.sav', 'rb'))
model_800 = pickle.load(open('../renyi_shannon_prollharm_800.sav', 'rb'))

kernel = [800,800]

In [7]:
%prun -s ncalls our_solution(y, 5, kernel, model_800)

In [36]:
%prun -s ncalls swgm_time(y, [512, 1024, 4096])

In [8]:
%prun librosa.stft(y, n_fft=512)

In [9]:
%prun np.fft.rfft(y)

In [18]:
a = [1,2,3]
b = [4,4,4]

In [19]:
np.dot(a,b)

24

In [20]:
%prun np.dot(a,b)

In [25]:
def uma_funcao(a,b):
    c = np.dot(a,b)
    d = np.prod(a)
    return c

In [27]:
test = %prun -r uma_funcao(a,b)

In [39]:
a = str(test)

In [43]:
test.dump_stats('teste.txt')

In [44]:
import pstats

In [45]:
b = ''

In [47]:
d = pstats.Stats('teste.txt', stream=b)